#### The purpose of notebook is to deploy a tensorflow model in an ec2 instance with EI accelerator using SageMaker

In [7]:
# For this notebook to run, choose environment Tensorflow 2.1 or less (e.g., Tensorflow 2.1 Python 3.7 GPU optimized)
# Note: TF only works with Py 2 and there no image with Py3.

In [5]:
import os
import sagemaker
from sagemaker import get_execution_role
# set up session
sagemaker_session = sagemaker.Session()
# IAM role
role = get_execution_role()
# region
region = sagemaker_session.boto_session.region_name

In [8]:
################################################
# Instructions to prepare the training python file:
################################################
# (1) Download mnist.py file from below path
# https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-python-sdk/tensorflow_script_mode_training_and_serving/mnist-2.py
# (2) Rename "mnist-2.py" to "mnist-2.py".
# (3) Copy the "mnist.py" into the  root folder on the left side bar of the sagemaker studio notebook.
# Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html#studio-ui-nav-bar for left side bar information.
# (4) Prettify the python source code using pygmentize (see the next cell).

In [ ]:
# prettify mnist.py file
!pygmentize 'mnist.py'

In [6]:
# S3 path for training data, copy the downloaded mnist.py file to this folder
training_data_uri = "s3://sagemaker-model-scripts/tensorflow/mnist"
training_data_uri = "s3://sagemaker-sample-data-{}/tensorflow/mnist".format(region)
print(training_data_uri)

s3://sagemaker-sample-data-us-east-1/tensorflow/mnist


In [9]:
# configure TF model entry point and environment info (instance count, instance type etc)
from sagemaker.tensorflow import TensorFlow

# TensorFlow model config
mnist_estimator2 = TensorFlow(
    entry_point="mnist.py",
    role=role,
    instance_count=2,
    instance_type="ml.m4.2xlarge",
    framework_version="2.1",
    py_version="py2",
    distribution={"parameter_server": {"enabled": True}},
)

In [11]:
# training the model
mnist_estimator2.fit(training_data_uri)

2022-07-12 07:21:26 Starting - Starting the training job...
2022-07-12 07:21:50 Starting - Preparing the instances for trainingProfilerReport-1657610486: InProgress
.........
2022-07-12 07:23:14 Downloading - Downloading input data...
2022-07-12 07:23:51 Training - Downloading the training image..2022-07-12 07:24:08,128 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2022-07-12 07:24:08,136 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-12 07:24:08,593 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2022-07-12 07:24:08,594 sagemaker_tensorflow_container.training INFO     Launching parameter server process
2022-07-12 07:24:08,594 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2022-07-12 07:24:08,706 sagemaker_tensorflow_container.training INFO     Launching worker process
2022-07-12 0

In [12]:
# deploying a Tensorflow model files using EI
predictor = mnist_estimator2.deploy(
initial_instance_count=1, # ec2 initial count
   instance_type='ml.m4.xlarge', # ec2 instance type
   accelerator_type='ml.eia2.medium') # accelerator type



AttributeError: The TensorFlow version 2.1 doesn't support EIA.

In [ ]:
%%time
# prediction
import numpy as np
random_input = np.random.rand(1, 1, 3, 3)
# predict for a sample input
prediction = predictor.predict({"inputs": random_input.tolist()})
# print prediction
print(prediction)

In [ ]:
# print endpoint
print(predictor.endpoint)

In [ ]:
import sagemaker
# delete the endpoint
predictor.delete_endpoint()